In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset

train_dataset = load_dataset("csv", data_files="train.csv")
dev_dataset = load_dataset("csv", data_files="validation.csv")

ModuleNotFoundError: No module named 'datasets'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("freedomking/mc-bert")


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_train=train_dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dev=dev_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "Request-Symptom", 1: "Inform-Symptom",2:"Request-Etiology",3:"Inform-Etiology",4:"Request-Basic_Information",5:"Inform-Basic_Information",6:"Request-Existing_Examination_and_Treatment",7:"Inform-Existing_Examination_and_Treatment",8:"Request-Drug_Recommendation",9:"Inform-Drug_Recommendation",10:"Request-Medical_Advice",11:"Inform-Medical_Advice",12:"Request-Precautions",13:"Inform-Precautions",14:"Diagnose",15:"Other"}
label2id = {"Request-Symptom": 0, "Inform-Symptom": 1,"Request-Etiology":2,"Inform-Etiology":3,"Request-Basic_Information":4,"Inform-Basic_Information":5,"Request-Existing_Examination_and_Treatment":6, "Inform-Existing_Examination_and_Treatment":7,"Request-Drug_Recommendation":8,"Inform-Drug_Recommendation":9, "Request-Medical_Advice":10, "Inform-Medical_Advice":11,"Request-Precautions":12,"Inform-Precautions":13,"Diagnose":14,"Other":15}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModel,AutoConfig,AutoModelForPreTraining

model = AutoModelForSequenceClassification.from_pretrained("freedomking/mc-bert", num_labels=16, id2label=id2label, label2id=label2id)

In [ ]:
training_args = TrainingArguments(
    output_dir="IMCS-DAC-mc-bert",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    label_names=None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_dev["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()